### Instalando spacy e fazendo download do corpus para português

In [1]:
!pip install -U spacy
!python -m spacy download pt_core_news_md
import spacy
import pt_core_news_md
import re

nlp = pt_core_news_md.load()


  Using cached spacy-3.4.0-cp38-cp38-win_amd64.whl (12.1 MB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached spacy_legacy-3.0.9-py2.py3-none-any.whl (20 kB)
  Using cached spacy_loggers-1.0.2-py3-none-any.whl (7.2 kB)
  Using cached thinc-8.1.0-cp38-cp38-win_amd64.whl (1.3 MB)
  Using cached blis-0.7.8-cp38-cp38-win_amd64.whl (6.6 MB)
  Attempting uninstall: blis
    Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.5
    Uninstalling thinc-7.4.5:
      Successfully uninstalled thinc-7.4.5
  Attempting uninstall: spacy-legacy
    Found existing installation: spacy-legacy 3.0.5
    Uninstalling spacy-legacy-3.0.5:
      Successfully uninstalled spacy-legacy-3.0.5
  Attempting uninstall: spacy
    Found existing installation: spacy 2.3.7
    Uninstalling spacy-2.3.7:
      Successfully uninstalled spacy-2.3.7


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 2.3.3 requires spacy<2.4.0, but you have spacy 3.4.0 which is incompatible.
pt-core-news-sm 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 3.4.0 which is incompatible.
pt-core-news-md 2.3.0 requires spacy<2.4.0,>=2.3.0, but you have spacy 3.4.0 which is incompatible.
es-core-news-sm 2.3.1 requires spacy<2.4.0,>=2.3.0, but you have spacy 3.4.0 which is incompatible.
en-core-web-sm 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 3.4.0 which is incompatible.
allennlp 2.8.0 requires spacy<3.2,>=2.1.0, but you have spacy 3.4.0 which is incompatible.
lda-classification 0.0.3 requires setuptools~=49.6.0, but you have setuptools 51.1.1 which is incompatible.
lda-classification 0.0.3 requires spacy~=2.3.1, but you have spacy 3.4.0 which is incompatible.
lda-classification 0.0.3 requires tomotopy~=0.9

  Attempting uninstall: pt-core-news-md
    Found existing installation: pt-core-news-md 2.3.0
    Uninstalling pt-core-news-md-2.3.0:
      Successfully uninstalled pt-core-news-md-2.3.0
[+] Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')


In [111]:
def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

def tokenizaFrase(frase, lower):
    frase = frase.replace('  ',' . ').replace(':',' ').replace('+',' . ').replace('#',' ').replace('(',' (').replace(')',') ').replace('[',' [').replace(']','] ')
    frase = replaceWhiteSpaces(frase.strip())
    if lower==1:
        frase=frase.lower()
    doc = nlp(frase)
    return doc
    
def isTokenFimChunk(pos):
    if pos != 'VERB' and pos != 'AUX' and pos != 'PUNCT' and pos != 'PRON' and pos != 'SPACE':    
        return False
    else:
        return True
    
def isTokenImportant(pos):
    if pos == 'NOUN' or pos == 'PROPN' or pos == 'ADJ':
        return True
    else:
        return False
    
def deveInserirToken(tamanhoTokens, pos):
    if tamanhoTokens!=0 or (tamanhoTokens==0 and (pos != 'ADP' and pos != 'DET' and pos != 'SCONJ' and pos != 'CCONJ')):
        return True
    else:
        return False
    
def deveManterToken(pos):
    if pos != 'DET' and pos != 'SCONJ' and pos != 'CCONJ' and pos != 'ADP':
        return True
    else:
        return False
    
def get_np(frase, lower):
        doc = tokenizaFrase(frase, lower)
        lista_chunks = []
        all_novo_chunk = []
        temNoun=0
        tokens_chunk = []
        for num, token in enumerate(doc):
            if not isTokenFimChunk(token.pos_):
                if deveInserirToken(len(tokens_chunk), token.pos_):
                    tokens_chunk.append(token)
                if isTokenImportant(token.pos_):
                    temNoun=1
            else:
                if len(tokens_chunk)!=0 and temNoun == 1:
                    novo_chunk = ''
                    deve_manter = 0
                    # retirando ultimos tokens do chunk se não forem importantes
                    for i in range(len(tokens_chunk)-1,-1,-1):
                        pos = tokens_chunk[i].pos_
                        texto = tokens_chunk[i].text
                        if deveManterToken(pos):
                            deve_manter = 1
                        if deve_manter==1:
                            novo_chunk = texto+' '+novo_chunk
                    all_novo_chunk.append(novo_chunk.strip())
                    lista_chunks.append(tokens_chunk)
                    tokens_chunk = []
                    temNoun = 0
                else:
                        tokens_chunk = []
                        temNoun = 0
        return all_novo_chunk


In [117]:
#ex = 'Há um único quadro de leitura aberto iniciado por códon de metionina de 1.458 nt no quadro com uma homeobox e uma repetição CAX, e o quadro de leitura aberto está previsto para codificar uma proteína de 51.659 daltons.'
ex = 'Data de Criação do Documento: 22/04/2014   Dispneia importante aos esforços + dor tipo peso no peito no esforço. Obeso, has, icc  c # cintilografia miocardica para avaliar angina.'
#ex = 'Em uso de insulina nph, metformina, sinvastatina 40mg 2 cp, carvedilol 6,25mg, losartana 50mg 2 x dia, AAS 100mg, clopidogrel 75mg. Tabagista há mais de 40 anos. Hoje fuma 8 cigarros por dia. Antes 2 carteiras/dia. Nega queixas.  O # LAboratorial 07/08/2013: HB: 12,9; Hematocrito 37,5; glicemia: 144; microalbuminuria: 2030,7 mg.  Cintilografia de perfusao miocardica agosto/13: Sem evidencias de isquemia miocardica, fç do VE preservada com hipocinesia septal.'
#ex='AVC EM JUGULAR ESQUERDA, SEM SINAIS FLOGÍSTICOS, COM SOROTERAPIA EM CURSO. RELATA ELIMINAÇÕES VESICAIS E INTESTINAIS. RIM TRANSPLANTADO.'

print('---Frase original:---\n')
print(ex)
print('\n\n---Chunks da frase:---\n')
print(get_np(ex, 0))


---Frase original:---

Há um único quadro de leitura aberto iniciado por códon de metionina de 1.458 nt no quadro com uma homeobox e uma repetição CAX, e o quadro de leitura aberto está previsto para codificar uma proteína de 51.659 daltons.


---Chunks da frase:---

['único quadro de leitura', 'códon de metionina de 1.458 nt no quadro com uma homeobox e uma repetição CAX', 'quadro de leitura', 'previsto', 'proteína de 51.659']
